In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, transpile, Aer, IBMQ, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from tqdm.notebook import tqdm

from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter
import qiskit.quantum_info as qi

from qc_grader.challenges.spring_2022.helpers import generate_disordered_tb_instruction

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 다체 양자 동역학

이번 연습에서는 격자 무질서와 입자-입자 상호작용을 알아보도록 하겠습니다. 비평형 상태에서 초기화된 닫힌 양자 다체 시스템은 자신의 동역학을 따라 열평형화(thermalization)라고 하는 평형 상태에 도달합니다. 이 것은 통계 역학 법칙의 결과로, 시간이 지남에 따라 뜨거운 한잔의 커피가 주변 온도에 맞춰 식는 것과 유사합니다.

하지만, 격자 무질서의 존재는 시스템이 에르고딕(ergodic) 열평형화 상태로 진화하는 것을 방해합니다. 무질서와 입자의 상호작용 간의 이러한 상호작용은 MBL(다체 국소화, Many-body localization)을 초래하고 시스템이 항상 초기 상태의 기억을 유지할 수 있도록 합니다. MBL에 대한 자세한 내용은 [1]을 참조하십시오.

임의의 격자점 에너지를 갖는 밀접 결합 해밀토니언을 다시 생각해 봅시다.

$$H_{\rm tb}/\hbar = \sum^{3}_{i=0} (X_i X_{i+1} + Y_i Y_{i+1}) + \sum_i \epsilon_i Z_i $$

연습 2에서 살펴본 것처럼 $\epsilon_i=W \rm{cos}(2\pi\beta i)$로 설정하여 이 시스템에서 무질서를 흉내낼 수 있습니다. 여기서 $W$는 무질서 강도를 $\beta$는 준결정 주기를 결정합니다. 유니테리 시간적 변화 이전에 여러 큐비트를 $|1\rangle$ 상태로 초기화하여 시스템에 여러 개의 입자를 추가할 수 있습니다. $H_{\rm tb}$에서 각 격자점은 하나의 입자만 차지할 수 있으므로 입자 반발(repulsion) 상호 작용이 발생합니다.

[1] https://arxiv.org/abs/1804.11065

열평형화 붕괴의 징후 중 하나는 **불균형(imbalance)** 의 출현입니다. 열평형화된 시스템에서 격자의 각 격자점은 정상 상태에 도달한 후 동일한 평균적으로 동일한 수의 입자가 차지할 것으로 예상 할 수 있습니다. 그러나 상호 작용 시스템에 무질서가 추가되면 각 격자점에서 관찰된 열 값에 편차가 발생합니다. 시스템 불균형은 다음 식을 사용하여 양으로 나타낼 수 있습니다.

$$\mathcal{I}= \langle (N_e-N_o)/(N_e+N_o) \rangle$$

여기서 $N_e$ 및 $N_o$는 시스템의 짝수 및 홀수 격자점의 밀도(population)입니다. 열평형화된 시스템의 경우 이 $\mathcal{I}=0$이고 시스템 무질서로 인해 이 값에서 편차가 발생합니다.

<div class="alert alert-block alert-danger">
<b>도전 질문 3a</b>
양자 상태의 불균형을 출력하는 함수 작성
</div>

In [ ]:
def get_imbalance(state):
    ###EDIT CODE BELOW
    ### HINT: MAKE SURE TO SKIP CALCULATING IMBALANCE OF THE |00...0> STATE
    imbalance_val=0
    
    
    ###DO NOT EDIT BELOW
    
    return imbalance_val

In [ ]:
## Grade and submit your solution
from qc_grader.challenges.spring_2022 import grade_ex3a

grade_ex3a(get_imbalance) 


다음으로, 양자 정보와 얽힘 엔트로피가 시스템에서 어떻게 성장하는지 생각해 보겠습니다. 얽힌 시스템에서 나머지 시스템에 대한 정보를 무시하면 부분계는 서로 다른 수 상태(number state, Fock state)의 혼합 상태(mixed state)가 됩니다. **von Neumann 엔트로피** 를 조사하여 부분계 $A$와 나머지 시스템 간의 얽힘 정도에 대한 통찰을 얻을 수 있습니다.

$$\mathcal{S}_{\rm vn}(\rho_A)= -\rm{tr}(\rho_A \rm{ln} \rho_A)$$

여기서 $\rho_A= \rm{tr} _{\bar{A}} \rho$는 시스템을 부분 대각합(trace out) 한 부분계 $A$를 설명하는 환산 밀도 행렬(reduced density matrix)입니다. 부분계 $A$가 나머지 시스템과 완전히 얽혀 있으면 $\mathcal{S}_ {\rm vn}(\rho_A) = \rm{ln}2$인 반면, 부분계가 주변 환경과 완전히 분리 가능한 경우(곱 상태 (product state), 분리 가능한 상태)인 경우 $\mathcal{S}_{\rm vn}(\rho_A)=0$ 입니다. 다음 예를 살펴봅시다.

In [ ]:
bell_state = qi.Statevector(np.array([0,1,1,0])/np.sqrt(2))

rho_0 = qi.partial_trace(bell_state,[1]) # We trace out qubit 1
rho_1 = qi.partial_trace(bell_state,[0]) # We trace out qubit 0

print('QB0 vn entropy: ', qi.entropy(rho_0, base=np.exp(1)))
print('QB1 vn entropy: ', qi.entropy(rho_1, base=np.exp(1)))

## 12 큐비트 체인에서 다체 양자 동역학 탐구

이번 연습에서는 연습 2에서 이미 살펴본 것과 동일한 격자 무질서 패턴을 사용하도록 하겠습니다.

In [ ]:
t = Parameter('t')

In [ ]:
num_qubits=12
deltas=[Parameter('delta_{:d}'.format(idx)) for idx in range(num_qubits)]
disorder_trot_step=generate_disordered_tb_instruction(t, deltas, num_qubits)

In [ ]:
# Here we define the disorder pattern

beta=(np.sqrt(5)-1)/2 # DO NOT EDIT
AA_pattern=np.cos(2*np.pi*beta*np.arange(num_qubits)) # DO NOT EDIT

<div class="alert alert-block alert-danger">
<b>도전 질문 3b</b>
큐비트 0, 4, 8을 $|1\rangle$로 설정하여 시스템을 초기화합니다.
</div>

In [ ]:
delta_t=0.1
time_steps=np.arange(0,21,2)

circuits={}
Ws=[1,4,10]

for W in Ws:
    disorders=W*AA_pattern
    
    circuits[W]=[]

    for n_steps in time_steps:

        qr = QuantumRegister(num_qubits)
        qc = QuantumCircuit(qr)

        ###EDIT CODE BELOW
        
        
        ###DO NOT EDIT BELOW     

        for _ in range(n_steps):
            qc.append(disorder_trot_step, [i for i in range(num_qubits)])
        
        if n_steps!=0:
            qc = qc.bind_parameters({t: delta_t})
            qc = qc.bind_parameters({deltas[idx]: disorders[idx] for idx in range(num_qubits)})

        circuits[W].append(qc)

<div class="alert alert-block alert-danger">
<b>도전 질문 3c</b>
다양한 무질서 강도에 대하여 서로 다른 변화 시간 단계에서 큐비트 0의 von Neumann 엔트로피를 계산합니다.
</div>

<div class="alert alert-block alert-danger">
<b>도전 질문 3d</b>
다양한 무질서 강도에 대하여 다른 변화 시간 단계에서 격자의 불균형을 계산합니다.
</div>

In [ ]:
from qiskit import transpile

# Use Aer's statevector simulator
from qiskit import Aer

# Run the quantum circuit on a statevector simulator backend
backend_sim = Aer.get_backend('statevector_simulator')

probability_densities={}
state_vector_imbalances={}
vn_entropies={}

for W in tqdm(Ws):
    probability_densities[W]=[]
    state_vector_imbalances[W]=[]
    vn_entropies[W]=[]
    
    for circ in circuits[W]:

        transpiled_circ=transpile(circ, backend_sim, optimization_level=3)

        job_sim = backend_sim.run(transpiled_circ)

        # Grab the results from the job.
        result_sim = job_sim.result()
        outputstate = result_sim.get_statevector(transpiled_circ, decimals=6)
        ps=[]
        for idx in range(num_qubits):
            ps.append(np.abs(qi.partial_trace(outputstate,[i for i in range(num_qubits) if i!=idx]))[1,1]**2)
        
        entropy=0
        ### EDIT CODE BELOW (extract the density matrix of qubit 0 by tracing out all other qubits)
        
        
        ###DO NOT EDIT BELOW 
        
        imbalance=0
        ### EDIT CODE BELOW 
        
        
        ###DO NOT EDIT BELOW
        
        
        vn_entropies[W].append(entropy)
        probability_densities[W].append(ps)
        state_vector_imbalances[W].append(imbalance)

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,5), facecolor='white', sharey=True)

for i,W in enumerate(Ws):
    ax=axs[i]
    ax.pcolormesh(np.arange(0,num_qubits,1), time_steps*delta_t ,probability_densities[W])
    ax.set_xlabel('Qubit index')
    ax.set_xticks(np.arange(1,num_qubits+1,1))

axs[0].set_ylabel('Time (1/J)')

plt.show()

In [ ]:
for W in Ws:
    plt.plot(time_steps*delta_t,vn_entropies[W], '--o', label='W={:d}'.format(W))

plt.xlabel(r'Time (1/J)')
plt.ylabel(r'$\mathcal{S}_{\rm vn}(\rho_0)$')
plt.legend()
plt.show()

In [ ]:
## Grade and submit your solution
from qc_grader.challenges.spring_2022 import grade_ex3b

grade_ex3b(vn_entropies)


In [ ]:
for W in Ws:
    plt.plot(time_steps*delta_t,state_vector_imbalances[W], '--o', label='W={:d}'.format(W))

plt.xlabel(r'Time (1/J)')
plt.ylabel(r'$\mathcal{I}$')
plt.legend()
plt.show()

In [ ]:
## Grade and submit your solution
from qc_grader.challenges.spring_2022 import grade_ex3c

grade_ex3c(state_vector_imbalances)


## 추가 정보

**한글화:** 김경훈, 신소영